In [0]:
SHOW CATALOGS;

In [0]:
USE CATALOG powerbi_poc;

In [0]:
DESCRIBE DETAIL hive_metastore.poc_db.nyc_taxi_raw;

In [0]:
CREATE SCHEMA IF NOT EXISTS POWERBI_POC.POC_SCHEMA;

In [0]:
CREATE TABLE powerbi_poc.poc_schema.nyc_taxi_raw
AS 
SELECT * FROM hive_metastore.poc_db.nyc_taxi_raw;

In [0]:
CREATE TABLE powerbi_poc.poc_schema.bi_taxi
AS 
SELECT * FROM hive_metastore.poc_db.bi_taxi;

In [0]:
USE SCHEMA poc_schema;

In [0]:
SHOW TABLES;

In [0]:
OPTIMIZE powerbi_poc.poc_schema.nyc_taxi_raw;
OPTIMIZE powerbi_poc.poc_schema.bi_taxi;

In [0]:
CREATE OR REPLACE TABLE powerbi_poc.poc_schema.bi_taxi
AS
SELECT
    -- Date parts
    YEAR(tpep_pickup_datetime)  AS year,
    MONTH(tpep_pickup_datetime) AS month,
    DAY(tpep_pickup_datetime)   AS day,

    -- Dimensions
    payment_type,
    VendorID AS vendor_id,

    CASE
        WHEN HOUR(tpep_pickup_datetime) BETWEEN 5  AND 11 THEN 'Morning'
        WHEN HOUR(tpep_pickup_datetime) BETWEEN 12 AND 16 THEN 'Afternoon'
        WHEN HOUR(tpep_pickup_datetime) BETWEEN 17 AND 21 THEN 'Evening'
        ELSE 'Night'
    END AS time_bucket,

    -- Passenger & distance
    passenger_count,
    trip_distance,

    -- 💰 Monetary fields
    fare_amount,
    extra,
    mta_tax,
    tip_amount,
    tolls_amount,
    improvement_surcharge,
    total_amount,

    -- 🗺 Mapping columns
    pickup_latitude,
    pickup_longitude,
    dropoff_latitude,
    dropoff_longitude

FROM powerbi_poc.poc_schema.nyc_taxi_raw;

In [0]:
DELETE FROM powerbi_poc.poc_schema.bi_taxi
WHERE
    trip_distance NOT BETWEEN 0.1 AND 50
    OR fare_amount NOT BETWEEN 2.5 AND 500
    OR total_amount < fare_amount
    OR passenger_count NOT BETWEEN 1 AND 6
    OR tip_amount NOT BETWEEN 0 AND 200
    OR extra NOT BETWEEN 0 AND 20
    OR mta_tax NOT BETWEEN 0 AND 1
    OR tolls_amount NOT BETWEEN 0 AND 200
    OR pickup_latitude NOT BETWEEN 40.5 AND 41.0
    OR pickup_longitude NOT BETWEEN -74.3 AND -73.6
    OR dropoff_latitude NOT BETWEEN 40.5 AND 41.0
    OR dropoff_longitude NOT BETWEEN -74.3 AND -73.6;

In [0]:
GRANT USAGE ON CATALOG powerbi_poc TO `partnership_poc_group_1`;
GRANT USAGE ON SCHEMA powerbi_poc.poc_schema TO `partnership_poc_group_1`;

GRANT SELECT ON TABLE powerbi_poc.poc_schema.nyc_taxi_raw TO `partnership_poc_group_1`;
GRANT SELECT ON TABLE powerbi_poc.poc_schema.bi_taxi TO `partnership_poc_group_1`;

In [0]:
ALTER GROUP partnership_poc_group_1 
ADD USER `ritwick.p@sigmoidanalytics.com`;

In [0]:
SHOW GRANTS ON TABLE powerbi_poc.poc_schema.bi_taxi;

In [0]:
REVOKE SELECT ON TABLE powerbi_poc.poc_schema.bi_taxi
FROM partnership_poc_group_1;

REVOKE SELECT ON TABLE powerbi_poc.poc_schema.nyc_taxi_raw
FROM partnership_poc_group_1;

In [0]:
SHOW GRANTS ON TABLE powerbi_poc.poc_schema.bi_taxi;

In [0]:
CREATE OR REPLACE VIEW powerbi_poc.poc_schema.bi_taxi_secure
AS
SELECT
    year,
    month,

    CASE
        WHEN current_user() = 'abishek.nair@sigmoidanalytics.com'
        THEN NULL
        ELSE day
    END AS day,

    payment_type,
    vendor_id,
    time_bucket,

    passenger_count,
    trip_distance,
    fare_amount,
    extra,
    mta_tax,
    tip_amount,
    tolls_amount,
    improvement_surcharge,
    total_amount,

    pickup_latitude,
    pickup_longitude,
    dropoff_latitude,
    dropoff_longitude

FROM powerbi_poc.poc_schema.bi_taxi
WHERE
    (
        is_member('partnership_poc_group_1')
        AND vendor_id = 1
        AND year = 2016
    )
    OR 
    (
        current_user() = 'abishek.nair@sigmoidanalytics.com'
        AND vendor_id = 2
    )
    OR
    (
        current_user() = 'ritwick.p@sigmoidanalytics.com'
        AND year = 2016
    );